<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

    
# Building a model with local grid refinement

*O.N. Ebbens, Artesia, 2022*

This notebook shows how `nlmod` can be used to create a model with local grid refinement.

### Contents<a name="TOC"></a>
1. [Create model](#create)
2. [Local grid refinement](#lgr)
3. [Run Model](#run)
3. [Visualise](#Visualise)

In [ ]:
import os

import flopy
import pandas as pd
import geopandas as gpd
import hydropandas as hpd
import matplotlib.pyplot as plt
import nlmod
import warnings

In [ ]:
print(f'nlmod version: {nlmod.__version__}')

nlmod.util.get_color_logger('INFO');

### [1. Create model](#TOC)<a name="create"></a>

Modflow 6 makes it possible to use locally refined grids. In `nlmod` you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See also figures below.

In [ ]:
# model settings vertex
model_ws = "model3"
model_name = "IJm_planeten"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
refine_shp_fname = os.path.abspath(os.path.join("data", "planetenweg_ijmuiden"))
levels = 2
extent = [95000.0, 105000.0, 494000.0, 500000.0]
delr = 100.0
delc = 100.0
steady_state = False
steady_start = True
transient_timesteps = 5
perlen = 1.0
start_time = "2015-1-1"
use_regis = True
regis_botm_layer = "MSz1"
use_geotop = True
add_northsea = True
starting_head = 1.0

In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=use_regis,
    regis_botm_layer=regis_botm_layer,
    use_geotop=use_geotop,
    cachedir=cachedir,
    cachename="combined_layer_ds.nc",
)

# create a model ds by changing grid of layer_model
ds = nlmod.to_model_ds(
    layer_model, model_name, model_ws, delr=delr, delc=delc
)

# add time discretisation
ds = nlmod.time.set_ds_time(
    ds,
    start_time=start_time,
    steady_state=steady_state,
    steady_start=steady_start,
    transient_timesteps=transient_timesteps,
    perlen=perlen,
)

### [2. Local grid refinement](#TOC)<a name="lgr"></a>

the code below applies a local grid refinement to the layer model. The local grid refinement is based on the shapefile 'planetenweg_ijmuiden.shp', which contains a line shape of the Planetenweg, and the levels, which is 2. This means that the model cells at the Planetenweg will get a size of 25 x 25m. 

In [ ]:
# use gridgen to create vertex grid
ds = nlmod.grid.refine(
    ds, refinement_features=[(refine_shp_fname, "line", levels)]
)

if add_northsea:
    ds = nlmod.read.rws.add_northsea(ds, cachedir=cachedir)

In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create ims
ims = nlmod.sim.ims(sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# Create discretization
disv = nlmod.gwf.disv(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)

In [ ]:
# voeg grote oppervlaktewaterlichamen toe
da_name = "rws_oppwater"
rws_ds = nlmod.read.rws.get_surface_water(
    ds, da_name, cachedir=ds.cachedir, cachename=da_name + ".nc"
)
ds.update(rws_ds)
ghb = nlmod.gwf.ghb(ds, gwf, da_name)

# surface level drain
ahn_ds = nlmod.read.ahn.get_ahn(ds, cachedir=ds.cachedir, cachename="ahn.nc")
ds.update(ahn_ds)

drn = nlmod.gwf.surface_drain_from_ds(ds, gwf, resistance=10.0)


# add constant head cells at model boundaries
ds.update(nlmod.grid.mask_model_edge(ds, ds["idomain"]))
chd = nlmod.gwf.chd(ds, gwf, chd="edge_mask", head="starting_head")

In [ ]:
# add knmi recharge to the model datasets
knmi_ds = nlmod.read.knmi.get_recharge(
    ds, cachedir=ds.cachedir, cachename="recharge"
)
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)

In [ ]:
ds

### [3. Write and Run](#TOC)<a name="run"></a>


In [ ]:
nlmod.sim.write_and_run(
    sim, ds, write_ds=True, nb_path="03_local_grid_refinement.ipynb"
)

### [4. Visualise](#TOC)<a name="visualise"></a>

Using the `ds` and `gwf` variables it is quite easy to visualise model data. Below the modelgrid together with the surface water is shown.

In [ ]:
plan_weg_gdf = gpd.read_file(refine_shp_fname + ".shp")

# plot modelgrid
ax = nlmod.plot.modelgrid(ds)
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.legend()

# plot zoomed modelgrid
ax = nlmod.plot.modelgrid(ds)
ax.set_title("Planetenweg")
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.set_xlim(100000, 103000)
ax.set_ylim(495000, 497500)
ax.legend()

The model dataset of a vertex model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension icell2d instead of the dimensions x and y. 

In [ ]:
ds

To plot the same rasters as for the previous model we can use the `nlmod.plot.data_array()` function.

In [ ]:
ds["vertices"] = nlmod.grid.get_vertices(ds)

fig, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2, figsize=(14, 11))

nlmod.plot.data_array(ds["ahn"], ds, ax=axes[0][0])
nlmod.plot.data_array(ds["botm"][0], ds, ax=axes[0][1])
nlmod.plot.data_array(ds["idomain"][0], ds, ax=axes[1][0])
nlmod.plot.data_array(ds["edge_mask"][0], ds, ax=axes[1][1])

fig, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2, figsize=(14, 11))
nlmod.plot.data_array(ds["bathymetry"], ds, ax=axes[0][0])
nlmod.plot.data_array(ds["northsea"], ds, ax=axes[0][1])
nlmod.plot.data_array(ds["kh"][1], ds, ax=axes[1][0])
nlmod.plot.data_array(ds["recharge"][0], ds, ax=axes[1][1])

We can save the entire model as a UGRID NetCDF-file. This can be opened in qgis, as a 'Mesh Layer'. For more information see https://docs.qgis.org/3.16/en/docs/user_manual/working_with_mesh/mesh_properties.html

In [ ]:
fname = os.path.join(ds.figdir, 'results.nc')
nlmod.gis.ds_to_ugrid_nc_file(ds, fname)

### [5. Compare with measurements](#TOC)<a name="measurements"></a>

We can download the BRO groundwater observation data and compare the model results with this data.

In [ ]:
fname_pklz = os.path.join(ds.cachedir, 'oc_bro.pklz')
if os.path.exists(fname_pklz):
    oc = pd.read_pickle(fname_pklz)
else:
    oc = hpd.read_bro(extent=ds.extent, name='BRO', tmin=ds.time.values.min(), tmax=ds.time.values.max(), )
    oc.to_pickle(fname_pklz)

In [ ]:
# get modellayers
oc['modellayer'] = oc.gwobs.get_modellayers(ds=ds)

In [ ]:
# get modelled head at measurement points
ds['heads'] = nlmod.gwf.get_heads_da(ds)
oc_modflow = hpd.read_modflow(oc, gwf, ds['heads'].values, ds.time.values)

In [ ]:
# add modelled head to measured heads
obs_list_map = []
for gld in oc.index:
    o = oc.loc[gld,'obs'].resample('D').last().sort_index()
    modelled = oc_modflow.loc[gld, 'obs']
    modelled = hpd.GroundwaterObs(modelled.rename(columns={0: 'values'}), name=f'{o.name}_mod_lay{oc.loc[gld,"modellayer"]}', x=o.x, y=o.y, 
                                  tube_nr=o.tube_nr+1,screen_top=o.screen_top, screen_bottom=o.screen_bottom, 
                                  tube_top=o.tube_top, monitoring_well=o.monitoring_well, source='MODFLOW', unit= 'm NAP',
                                  ground_level=o.ground_level, metadata_available=o.metadata_available)
    obs_list_map.append(o)
    obs_list_map.append(modelled)

oc_map = hpd.ObsCollection.from_list(obs_list_map, name='meting+model')

# create interactive map
oc_map.plots.interactive_map(os.path.join(ds.figdir, 'iplots'))